In [1]:
!nvidia-smi

Sun Apr 21 03:05:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  |   00000000:41:00.0 Off |                  N/A |
| 23%   27C    P8              9W /  250W |       1MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/whnhch/cta.git

fatal: destination path 'cta' already exists and is not an empty directory.


In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: u1472329 (whanhee). Use `wandb login --relogin` to force relogin


True

In [4]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
import numpy as np
import sys
import random
from cta.models import TabCSE, TabCSEForClassification
import torch.nn as nn
import torch
import json

lr=1e-4
weight_decay=1e-4
epochs=4
bs=128
model_name='bert-base-uncased'
max_seq=256
temp=0.2

In [5]:
wandb.init(
    project="nlp-2024-tabcse",
    config={
        "batch_size": bs,
        "learning_rate": epochs,
        "dataset": "viznet",
    },
)

cat: /sys/module/amdgpu/initstate: No such file or directory
ERROR:root:Driver not initialized (amdgpu not found in modules)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

In [8]:
df1=pd.read_csv('sato_cv_0.csv')
df2=pd.read_csv('sato_cv_1.csv')
df3=pd.read_csv('sato_cv_2.csv')
df4=pd.read_csv('sato_cv_3.csv')
df5=pd.read_csv('sato_cv_4.csv')

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
f1_scores = []

In [11]:
from cta.utils.data import ViznetDataset

train_dataset_1=ViznetDataset(df1['data'].to_list(),df1['class_id'].to_list(),max_seq,tokenizer)

In [12]:
train_dataset_2=ViznetDataset(df2['data'].to_list(),df2['class_id'].to_list(),max_seq,tokenizer)

In [13]:
train_dataset_3=ViznetDataset(df3['data'].to_list(),df3['class_id'].to_list(),max_seq,tokenizer)

In [14]:
train_dataset_4=ViznetDataset(df4['data'].to_list(),df4['class_id'].to_list(),max_seq,tokenizer)

In [15]:
train_dataset_5=ViznetDataset(df5['data'].to_list(),df5['class_id'].to_list(),max_seq,tokenizer)
train_datasets=[train_dataset_1, train_dataset_2,train_dataset_3, train_dataset_4, train_dataset_5]

In [16]:
from sklearn.metrics import f1_score

loss_fct = nn.CrossEntropyLoss()

for fold_idx in range(5):
    print(f"Fold {fold_idx + 1}")

    datasets = train_datasets[:fold_idx] + train_datasets[fold_idx + 1:]
    print(f"start datasetify")
    
    print(f"start make data as tensor")
    input_ids = (datasets[0].x['input_ids']+datasets[1].x['input_ids']+datasets[2].x['input_ids']+datasets[3].x['input_ids'])
    input_ids = torch.tensor(input_ids)
    attention_masks = (datasets[0].x['attention_mask']+datasets[1].x['attention_mask']+datasets[2].x['attention_mask']+datasets[3].x['attention_mask'])
    attention_masks = torch.tensor(attention_masks)
    y = (datasets[0].y+datasets[1].y+datasets[2].y+datasets[3].y)
    y = torch.tensor(y)

    train_dataset = torch.utils.data.TensorDataset(input_ids,attention_masks, y)

    # Create a DataLoader
    dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    
    model=TabCSE(model_name,temp)
    checkpoint_path = 'tabcse_checkpoint.pth'
    state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict['model_state_dict'])
    
    model_cls=TabCSEForClassification(model, 78)
    optimizer = torch.optim.Adam(model_cls.parameters(), lr=lr)    
    model_cls.to(device)
    
    # Train the model
    model_cls.train()
    print(f"start trainig")
    for epoch in range(epochs):
        epoch_loss=0.0

        for idx, batch in enumerate(dataloader):
          x_input_ids, x_attention_mask, y = batch
          x_input_ids, x_attention_mask, y = x_input_ids.to(device), x_attention_mask.to(device), y.to(device)
          logits = model_cls(x_input_ids, x_attention_mask)

          loss = loss_fct(logits, y)
          epoch_loss += loss.item()

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (idx+1) % 100 == 0:
              average_loss = epoch_loss / 100
              print("Epoch [{}/{}], Iteration [{}/{}], Average Loss: {:.4f}".format(epoch + 1, epochs, idx+ 1, len(dataloader), average_loss))
              epoch_loss = 0.0
              wandb.log({"viznet_loss": average_loss})

    model_cls.eval()

    val_datasets=train_datasets[fold_idx]

    input_ids = torch.tensor(val_datasets.x['input_ids'])
    attention_masks = torch.tensor(val_datasets.x['attention_mask'])
    y = torch.tensor(val_datasets.y)

    test_dataset = torch.utils.data.TensorDataset(input_ids,attention_masks, y)
    dataloader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

    predictions = []
    ground_truth = []
    with torch.no_grad():
      for idx, batch in enumerate(dataloader):
          x_input_ids, x_attention_mask, y = batch
          x_input_ids, x_attention_mask, y = x_input_ids.to(device), x_attention_mask.to(device), y.to(device)

          logits = model_cls(x_input_ids, x_attention_mask)
          batch_predictions = logits.argmax(dim=1).cpu().numpy()

          predictions.extend(batch_predictions)
          ground_truth.extend(y.cpu().numpy())
    # Calculate F1 score and append to list
    f1 = f1_score(ground_truth, predictions, average='macro')
    wandb.log({"macro_f1": f1})
    f1_scores.append(f1)
    f1_ = f1_score(ground_truth, predictions, average='micro')
    wandb.log({"micro_f1": f1_})

    # Calculate and print the average F1 score
    avg_f1_score = np.mean(f1_scores)
    print(f"Average F1 Score: {avg_f1_score}")

Fold 1
start datasetify
start make data as tensor


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


start trainig


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 10.90 GiB of which 37.12 MiB is free. Including non-PyTorch memory, this process has 10.86 GiB memory in use. Of the allocated memory 10.66 GiB is allocated by PyTorch, and 44.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)